# This notebook is used to plot species divergence times for *Quercues* ser. *Virentes*.
Divergence times were computed used BPP Algorithm 00. The fixed tree used in the Algorithm 00 analysis is the same used here for plotting divergence times.

In [55]:
# import necessary packages for plotting
import toytree
import toyplot
import numpy as np

In [2]:
toytree.__version__

'0.1.18'

In order to create a baseline tree, I write out a newick tree structure.

In [4]:
newick = "((bran,fusi),((sagr, oleo),(virg,(mini,gemi))));"
tre = toytree.tree(newick)

tre.draw(node_labels=True)

(<toyplot.canvas.Canvas at 0x11891def0>,
 <toyplot.coordinates.Cartesian at 0x11891df60>)

gemi mini virg oleo sagr fusi bran idx: 0
name: gemi
dist: 1
support: 100
height: 0 0 idx: 1
name: mini
dist: 1
support: 100
height: 0 1 idx: 2
name: virg
dist: 1
support: 100
height: 1 2 idx: 3
name: oleo
dist: 1
support: 100
height: 1 3 idx: 4
name: sagr
dist: 1
support: 100
height: 1 4 idx: 5
name: fusi
dist: 1
support: 100
height: 2 5 idx: 6
name: bran
dist: 1
support: 100
height: 2 6 idx: 7
name: 7
dist: 1
support: 100
height: 1 7 idx: 8
name: 8
dist: 1
support: 100
height: 2 8 idx: 9
name: 9
dist: 1
support: 100
height: 2 9 idx: 10
name: 10
dist: 1
support: 100
height: 3 10 idx: 11
name: 11
dist: 1
support: 100
height: 3 11 idx: 12
name: 12
dist: 0
support: 100
height: 4 12

I create a dictionary for the species divergence times. The times correspond with the node on the tree they belond to. Nodes 0-6 are leaf nodes, therefore they are not assigned a divergence time value. 

In [5]:
div_dict = {
#     0:'',
#     1:'',
#     2:'',
#     3:'',
#     4:'',
#     5:'',
#     6:'',
    7:'3.405',
    8:'6.167',
    9:'3.525',
    10:'10.293',
    11:'7.585',
    12:'11.190'
}

The tree is then traversed in order to assign each node a distance value. Note: in the tree structure, node heights are immutable. Therefore, node distance is used to assign relative branch length values. The branch lengths are being modified so that the tree divergence times are able to be depicted on the tree, both with node labels and spatially in the tree structure.

In [6]:
for node in tre.treenode.traverse():
    #print(node.idx, node.height, node.dist)
    if node.idx in div_dict:
        height = div_dict[node.idx]
        node.h = height
    else:
        node.h = 0
        
        
for node in tre.treenode.traverse():
    #print(node.idx, node.h, node.dist)
    if not node.is_root():
        node.dist = float(node.up.h) - float(node.h)
    print(node.idx, node.dist, node.h)

12 0.0 11.190
11 3.6049999999999995 7.585
10 0.8970000000000002 10.293
6 7.585 0
5 7.585 0
9 6.767999999999999 3.525
8 4.1259999999999994 6.167
4 3.525 0
3 3.525 0
2 6.167 0
7 2.762 3.405
1 3.405 0
0 3.405 0


The tree is drawn using the newly modified edge lengths. Compared to the original tree, using only the underlying newick structure, branch lengths now reflect the estimated species divergence times.

In [7]:
tre.draw(use_edge_lengths=True);

gemi mini virg oleo sagr fusi bran

To plot the actual divergence time values on the tree, a new feature is added to all of the node objects. This feature is "age" and is added by traversing the tree and adding both the new "age" feature, and its associtated value using the divergence time dictionary.

In [8]:
for node in tre.treenode.traverse():
    node.add_feature("age", div_dict.get(node.idx))

In [9]:
# confirming that the age and divergence times values match up between values assigned and dictionary values
for node in tre.treenode.traverse():
    print(node.idx, node.age, div_dict.get(node.idx))

12 11.190 11.190
11 7.585 7.585
10 10.293 10.293
6 None None
5 None None
9 3.525 3.525
8 6.167 6.167
4 None None
3 None None
2 None None
7 3.405 3.405
1 None None
0 None None


The next tree drawn is one that plots the node ages on each of the divergence node points. The names of the leaf nodes and the edge lengths are not yet formatted.

In [10]:
tre.draw(
    node_labels='age',
    use_edge_lengths=False,
    node_labels_style={"font-size": "9px"},
    width=600,
    height=400
)

(<toyplot.canvas.Canvas at 0x1189675f8>,
 <toyplot.coordinates.Cartesian at 0x1189847f0>)

gemi mini virg oleo sagr fusi bran idx: 7
name: 7
dist: 2.7620
support: 100
height: 3.4050
age: 3.405 3.405 idx: 8
name: 8
dist: 4.1260
support: 100
height: 6.1670
age: 6.167 6.167 idx: 9
name: 9
dist: 6.7680
support: 100
height: 3.5250
age: 3.525 3.525 idx: 10
name: 10
dist: 0.8970
support: 100
height: 10.2930
age: 10.293 10.293 idx: 11
name: 11
dist: 3.6050
support: 100
height: 7.5850
age: 7.585 7.585 idx: 12
name: 12
dist: 0.0000
support: 100
height: 11.1900
age: 11.190 11.190

Then another dictionary is made for renaming the leaf node names. Each abbreviation is given a corresponding full species name and italicized. 

In [11]:
species_dict = {
    "bran":"<i>Q. brandegeei</i>",
    "fusi":"<i>Q. fusiformis</i>",
    "sagr":"<i>Q. sagraeana</i>",
    "oleo":"<i>Q. oleoides</i>",
    "virg":"<i>Q. virginiana</i>",
    "mini":"<i>Q. minima</i>",
    "gemi":"<i>Q. geminata</i>"
}

Then the final tree is drawn! Tip labels are changed to the corresponding species name (the actual node names are not changed for the treenode objects). Now all the edge lengths and node labels correspond with their divergence time in million years. 

# This is the final divtime tree for the denovo BPP Analysis

In [58]:
canvas, axes = tre.draw(
    tip_labels=[species_dict[i] for i in tre.get_tip_labels()],
    use_edge_lengths=True,
    node_labels='age',
    node_colors=toytree.colors[6],
    node_labels_style={"font-size": "9px"},
    node_markers="r2.5x1.25",
    node_sizes=12,
    width=800,
    height=400,
    scalebar=True,
)

Q. geminata Q. minima Q. virginiana Q. oleoides Q. sagraeana Q. fusiformis Q. brandegeei idx: 7
name: 7
dist: 2.7620
support: 100
height: 3.4050
age: 3.405 3.405 idx: 8
name: 8
dist: 4.1260
support: 100
height: 6.1670
age: 6.167 6.167 idx: 9
name: 9
dist: 6.7680
support: 100
height: 3.5250
age: 3.525 3.525 idx: 10
name: 10
dist: 0.8970
support: 100
height: 10.2930
age: 10.293 10.293 idx: 11
name: 11
dist: 3.6050
support: 100
height: 7.5850
age: 7.585 7.585 idx: 12
name: 12
dist: 0.0000
support: 100
height: 11.1900
age: 11.190 11.190 0.0 1.6 3.2 4.8 6.4 8.0 9.6 11.2

In [60]:
import toyplot.pdf
toyplot.pdf.render(canvas, "denovo_divtime-tree-plot.pdf")

to create the reference BPP analysis divtime tree, I am using the same newick structure

In [31]:
ref = toytree.tree(newick)
ref.draw(node_labels=True)

(<toyplot.canvas.Canvas at 0x118a4f6a0>,
 <toyplot.coordinates.Cartesian at 0x118a4f438>)

gemi mini virg oleo sagr fusi bran idx: 0
name: gemi
dist: 1
support: 100
height: 0
age: 0 idx: 1
name: mini
dist: 1
support: 100
height: 0
age: 1 idx: 2
name: virg
dist: 1
support: 100
height: 1
age: 2 idx: 3
name: oleo
dist: 1
support: 100
height: 1
age: 3 idx: 4
name: sagr
dist: 1
support: 100
height: 1
age: 4 idx: 5
name: fusi
dist: 1
support: 100
height: 2
age: 5 idx: 6
name: bran
dist: 1
support: 100
height: 2
age: 6 idx: 7
name: 7
dist: 1
support: 100
height: 1
age: 7 idx: 8
name: 8
dist: 1
support: 100
height: 2
age: 8 idx: 9
name: 9
dist: 1
support: 100
height: 2
age: 9 idx: 10
name: 10
dist: 1
support: 100
height: 3
age: 10 idx: 11
name: 11
dist: 1
support: 100
height: 3
age: 11 idx: 12
name: 12
dist: 0
support: 100
height: 4
age: 12

In [32]:
ref_div_dict = {
    #     0:'',
#     1:'',
#     2:'',
#     3:'',
#     4:'',
#     5:'',
#     6:'',
    7:'9.063',
    8:'11.377',
    9:'7.095',
    10:'14.895',
    11:'10.480',
    12:'16.166'
}

In [36]:
for node in ref.treenode.traverse():
    #print(node.idx, node.height, node.dist)
    if node.idx in ref_div_dict:
        height = ref_div_dict[node.idx]
        node.h = height
    else:
        node.h = 0

for node in ref.treenode.traverse():
    #print(node.idx, node.h, node.dist)
    if not node.is_root():
        node.dist = float(node.up.h) - float(node.h)
    print(node.idx, node.dist, node.h)

12 0.0 16.166
11 5.686 10.480
10 1.2710000000000008 14.895
6 10.48 0
5 10.48 0
9 7.8 7.095
8 3.517999999999999 11.377
4 7.095 0
3 7.095 0
2 11.377 0
7 2.314 9.063
1 9.063 0
0 9.063 0


In [37]:
ref.draw(use_edge_lengths=True)

(<toyplot.canvas.Canvas at 0x118a36470>,
 <toyplot.coordinates.Cartesian at 0x118a361d0>)

gemi mini virg oleo sagr fusi bran

In [42]:
for node in ref.treenode.traverse():
    node.add_feature("age", ref_div_dict.get(node.idx))

# This is the tree for the reference BPP divtimes

In [61]:
canvas_ref, axes = ref.draw(
    tip_labels=[species_dict[i] for i in ref.get_tip_labels()],
    use_edge_lengths=True,
    node_labels='age',
    node_colors=toytree.colors[6],
    node_labels_style={"font-size": "9px"},
    node_markers="r2.5x1.25",
    node_sizes=12,
    width=800,
    height=400,
    scalebar=True, 
)

Q. geminata Q. minima Q. virginiana Q. oleoides Q. sagraeana Q. fusiformis Q. brandegeei idx: 7
name: 7
dist: 2.3140
support: 100
height: 9.0630
age: 9.063 9.063 idx: 8
name: 8
dist: 3.5180
support: 100
height: 11.3770
age: 11.377 11.377 idx: 9
name: 9
dist: 7.8000
support: 100
height: 7.0950
age: 7.095 7.095 idx: 10
name: 10
dist: 1.2710
support: 100
height: 14.8950
age: 14.895 14.895 idx: 11
name: 11
dist: 5.6860
support: 100
height: 10.4800
age: 10.480 10.480 idx: 12
name: 12
dist: 0.0000
support: 100
height: 16.1660
age: 16.166 16.166 0.0 2.3 4.6 6.9 9.2 11.5 13.9 16.2

In [62]:
import toyplot.pdf
toyplot.pdf.render(canvas_ref, "ref_divtime-tree-plot.pdf")